确定 M（测量算子）和 $\vert\psi\rangle$（量子态）  
每个可观测量对应一个厄米算子（Hermitian operator） 
如能量对应哈密顿量H  
自旋z分量对应泡利算子（若要测量某个可观测量（如自旋z）,需构造对应的测量算子M

In [1]:
import time
import traceback

import quairkit as qkit
from quairkit.database import *
from quairkit.loss import *
from quairkit.qinfo import *

qkit.set_dtype("complex128")

泡利算符X[[0 1][1 0]]的本征值为±1，对应的本征态有两个(就是两个eigenvector)，投影算子是把任意量子态投影到某个特定本征态上的工具，P+会把态投影到|+>上，为$P_+ = |+\rangle\langle +|$（投影到 $|+\rangle$ 的算子）和相应的-  
投影是把任意态投影到这两个本征态上，只需要两个投影算符  

∣00⟩=∣0⟩⊗∣0⟩=[1 0 0 0]
任何 2-qubit 量子态都可以表示为这四个基态的线性组合（叠加态）。

In [2]:
pvm = pauli_str_povm("x")  
#生成与泡利 X 算符对应的 PVM 测量算子集合。

print(f"Projection-valued measure: \n{pvm}")

Projection-valued measure: 
tensor([[[ 0.5000+0.j,  0.5000+0.j],
         [ 0.5000+0.j,  0.5000+0.j]],

        [[ 0.5000+0.j, -0.5000+0.j],
         [-0.5000+0.j,  0.5000+0.j]]])


测量第 0 位 qubit 后，若量子态被约束到这些本征态，会呈现特定的数学特征。  
以结果 0 的塌缩态为例（对应本征态 $|+\rangle$），其矩阵应满足：第 0 位 qubit 的$|0\rangle$和$|1\rangle$分量完全对称。  
左上角（$|00\rangle, |01\rangle$行 / 列）：对应第 0 位为$|0\rangle$的子空间右下角（$|10\rangle, |11\rangle$行 / 列）：对应第 0 位为$|1\rangle$的子空间  
这两个子空间分量完全相等

将 4×4 矩阵按 “第 0 位 qubit（A）” 和 “第 1 位 qubit（B）” 的基矢拆分，得到 4 个 2×2 子矩阵：   
[[ρ00 ρ01]  
 [ρ10 ρ11]]  


ρ AB = ∑i,j,k,l∈{0,1}  ρ ij,kl  ∣ij⟩⟨kl∣  
$\rho_{ij,kl} = \langle ij|\rho_{AB}|kl\rangle$是矩阵元    
当对 B 求迹时，实际是将 “B=0” 和 “B=1” 的子空间中A 的相同分量相加  
A=0 的分量：来自$\rho_{00}$（B=0 时 A=0 的信息）和$\rho_{01}$中 B=1 时 A=0 的信息  


副对角线:
它们描述的是 A 和 B 之间的 “交叉关联”，而部分迹$\text{Tr}_B$的目的是 “消除 B 的信息，只保留 A 的独立状态”，这些交叉项会在对 B 求迹时被 “正交性” 抵消。

部分迹$\text{Tr}_B(\rho_{AB})$的严格定义是：$\text{Tr}_B(\rho_{AB}) = \sum_{b \in \{0,1\}} \langle b|_B \rho_{AB} |b\rangle_B$  
计算$\langle 0|_B \rho_{AB} |0\rangle_B$时，由于 B 的正交性，只有 B=|0⟩的子空间（即$\rho_{00}$）会保留，其他分块（$\rho_{01},\rho_{10},\rho_{11}$）因包含 B=|1⟩的成分而被消去，结果为$\rho_{00}$  


量子中，密度矩阵的分块$\rho_{00}$对应 “A 的状态与 B=|0⟩的关联”，$\rho_{11}$对应 “A 的状态与 B=|1⟩的关联”。对 B 求部分迹时，“相加” 本质是对 B 的所有可能状态（|0⟩和 | 1⟩）求和，得到 A 的总概率分布。

对第 1 位（B）求部分迹的数学操作是：将与第 1 位基矢相关的子矩阵按 “迹” 规则求和，即保留第 0 位的信息，对第 1 位的基矢维度求和。<就是按照主对角线规则求(因为是0 1)>

仅测量第 0 位时，第 1 位的状态不会被直接改变，但会通过量子关联间接受到影响（体现在坍缩态的矩阵元中）。

坍缩后的量子态需要完整描述两个量子比特的状态及其关联：第 0 位：状态确定（0 或 1），不再有叠加性；第 1 位：仍可能处于叠加态（如$a|0\rangle + b|1\rangle$），保留量子特性；两者的关联：第 1 位的状态可能依赖于第 0 位的测量结果（如量子纠缠的残余影响）

In [3]:
op = Measure()  # 计算基测量，M_m = |m⟩⟨m|  
#测量算子是计算基的投影  
op = Measure("x")  # 用泡利1X作为本征基测量 
op = Measure(pvm)  # 用自定义PVM张量初始化  

op = Measure(pvm)
# 2-qubit就是4维  
# ρ=∣ψ⟩⟨ψ∣ rank=2是混合态，rank=1才是纯态  
#∣ψ⟩=∑ i,j∈{0,1} cij ∣ij⟩  (cij是概率幅)  

rho = random_state(num_qubits=2, rank=2)  # 生成2-qubit随机态密度矩阵  
prob = op(rho, qubits_idx=[0])  # 测量第0位qubit
print(rho)
print("测量结果概率分布:", prob)  
prob, collapsed_state = op(rho, qubits_idx=[0], keep_state=True)
print("塌缩后的量子态:", collapsed_state)
#结果第0位量子比特可能是0也可能是1


-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2, 2]
 System sequence: [0, 1]
[[ 0.25+0.j   -0.16+0.13j  0.14+0.09j  0.09-0.21j]
 [-0.16-0.13j  0.18+0.j   -0.02-0.14j -0.12+0.07j]
 [ 0.14-0.09j -0.02+0.14j  0.16+0.j    0.05-0.17j]
 [ 0.09+0.21j -0.12-0.07j  0.05+0.17j  0.41+0.j  ]]
-----------------------------------------------------

测量结果概率分布: tensor([0.5279, 0.4721])
塌缩后的量子态: 
-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [2]

 # 0:
[[ 0.33+0.j   -0.02-0.05j  0.33+0.j   -0.02-0.05j]
 [-0.02+0.05j  0.17+0.j   -0.02+0.05j  0.17+0.j  ]
 [ 0.33+0.j   -0.02-0.05j  0.33+0.j   -0.02-0.05j]
 [-0.02+0.05j  0.17+0.j   -0.02+0.05j  0.17+0.j  ]]
 # 1:
[[ 0.06+0.j   -0.09+0.02j -0.06+0.j    0.09-0.02j]
 [-0.09-0.02j  0.44+0.j    0.09+0.02j -0.44+0.j  ]
 [-0.06+0.j    0.09-0.02j  0.06+0.j   -0.09+0.02j]
 [ 0.09+0.02j -0.44+0.j   -0.09-0.02j  0.44+0.

In [4]:
x = "1"
prob, collapsed_state = op(
    rho, qubits_idx=[0], keep_state=True, desired_result=x
)  # return the second outcome
print(
    f"The probability for obtaining outcome {x} is {prob}, with outcome state",
    collapsed_state,
)

The probability for obtaining outcome 1 is tensor([0.4721]), with outcome state 
-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2, 2]
 System sequence: [0, 1]
[[ 0.06+0.j   -0.09+0.02j -0.06+0.j    0.09-0.02j]
 [-0.09-0.02j  0.44+0.j    0.09+0.02j -0.44+0.j  ]
 [-0.06+0.j    0.09-0.02j  0.06+0.j   -0.09+0.02j]
 [ 0.09+0.02j -0.44+0.j   -0.09-0.02j  0.44+0.j  ]]
-----------------------------------------------------



↓ 可以同时获取测量结果的概率分布和对应结果的坍缩态

In [5]:
rho = random_state(num_qubits=1, rank=2)
prob = rho.measure(pvm)  # same as Measure(pvm)(rho)
print("The probability distribution of outcome", prob)
prob, collapsed_state = rho.measure(
    pvm, keep_state=True
)  # same as Measure(pvm)(rho, keep_state=True)
print("The collapsed state for each outcome is", collapsed_state)

The probability distribution of outcome tensor([0.3437, 0.6563])
The collapsed state for each outcome is 
-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
 # 1:
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
-----------------------------------------------------



pvm的测量算子是投影算子，POVM 的算子 并不一定是投影算子

In [8]:
start_time = time.time()
prob = rho.measure(pvm)
print(f"Time for measuring with pvm: {time.time() - start_time:.10f}s")
#使用pvm对量子态rho进行测量

start_time = time.time()
prob = rho.measure(pvm, is_povm=True)
print(f"Time for measuring with povm: {time.time() - start_time:.10f}s")
#使用povm对量子态rho进行测量
try:
    rho.measure(pvm, is_povm=True, 
                # keep_state=True
                # povm无法明确获得测量后态
                )
except ValueError:
    traceback.print_exc()

Time for measuring with pvm: 0.0010001659s
Time for measuring with povm: 0.0000000000s


In [ ]:
op = Measure(["x", "y", "z"]) 
## 用 Pauli 字符串初始化批量测量：x、y、z 基分别对应 1 组 PVM

#手动构造pvm，效果和上面一样
list_pvm = pauli_str_povm(["x", "y", "z"])  
print(f"The tensor shape of the measurement: {list_pvm.shape}")
op = Measure(list_pvm)


In [ ]:
#单量子态测量
prob, collapsed_state = op(rho, keep_state=True)
print(
    "The measured states for the first batch is",
    collapsed_state[0],
    f"with prob distribution {prob[0]}",
)
#结果
'''
用密度矩阵表示的量子态
单量子比特系统维度为2
测量的是第0号量子比特
单个量子态测量后，可能的坍缩结果数量
'''

The measured states for the first batch is 
-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
-----------------------------------------------------
 with prob distribution 0.34365089664479753


In [16]:
# 三个量子态+一组pvm(三个)
#每个量子态测量后都得到一组pvm的概率分布
batch_size = 3

list_rho = random_state(num_qubits=1, #生成的量子态只有一个
                        size=batch_size)
#一共生成三个独立的量子态，每个都是单1qubit系统

prob, collapsed_state = op(list_rho, keep_state=True)
print(
    "The measured states for the first batch is",
    collapsed_state[0],
    f"with prob distribution {prob[0]}",
    #测量了三个量子态，但是只输出了第一个的结果
)

The measured states for the first batch is 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[0.53+0.47j 0.53+0.47j]
 # 1:
[ 0.69+0.17j -0.69-0.17j]
-----------------------------------------------------
 with prob distribution tensor([0.2534, 0.7466])


Sampled measurements量子测量结果采样

In [18]:
batch_size = 3
print(f"{batch_size} probability distributions are\n", prob)

print(f"\nThe outcomes of quantum measurements:\n{prob_sample(prob)}")
print(
    f"\nThe outcomes of quantum measurements with the decimal system of dictionary system:\n",
    prob_sample(prob, binary=False),
)
print(
    f"\nThe outcomes of quantum measurements in proportion:\n",
    prob_sample(prob, proportional=True),
)
# prob_sample默认采样次数（即shots）通常是1024  
#是基于概率分布进行随机采样，每次调用会生成不同的随机数决定采样结果，
# 有理论值波动
#如果要固定，需要固定随机数生成的种子

3 probability distributions are
 tensor([[0.2534, 0.7466],
        [0.6040, 0.3960],
        [0.8568, 0.1432]])

The outcomes of quantum measurements:
{'0': tensor([228, 610, 866]), '1': tensor([796, 414, 158])}

The outcomes of quantum measurements with the decimal system of dictionary system:
 {'0': tensor([263, 608, 898]), '1': tensor([761, 416, 126])}

The outcomes of quantum measurements in proportion:
 {'0': tensor([0.2578, 0.5986, 0.8818]), '1': tensor([0.7422, 0.4014, 0.1182])}


In [19]:
qkit.print_info()


---------VERSION---------
quairkit: 0.4.0
torch: 2.7.1+cpu
numpy: 1.26.4
scipy: 1.15.3
matplotlib: 3.10.3
---------SYSTEM---------
Python version: 3.10.18
OS: Windows
OS version: 10.0.22000
---------DEVICE---------
CPU: Intel64 Family 6 Model 154 Stepping 3, GenuineIntel
